In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

In [ ]:
# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# GPT-2 does not have a padding token by default, so we add one
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Resize token embeddings to accommodate the new padding token
model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50258, 768)

In [ ]:
from datasets import load_dataset
from datasets import Dataset

In [ ]:
dataset = load_dataset('kalyan003/Question_Answer_Dataset')
dataset = dataset['train']
dataset

README.md:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Only_Question_answer.txt:   0%|          | 0.00/3.09M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5753 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 5753
})

In [ ]:
# Slice to keep only the last 1000 rows
testing = dataset[0:500]

# Create a new Dataset object from the sliced data
testing_dataset = Dataset.from_dict({
    'text': testing['text']
})

# Slice to keep only the last 1000 rows
traning = dataset[:1000]

# Create a new Dataset object from the sliced data
traning_dataset = Dataset.from_dict({
    'text': traning['text']
})

In [ ]:
traning_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [ ]:
testing_dataset

Dataset({
    features: ['text'],
    num_rows: 500
})

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'],
                     truncation=True,
                     padding='max_length',
                     max_length=128)

# Tokenize training dataset
tokenized_train = traning_dataset.map(tokenize_function, batched=True)

# Tokenize testing dataset
tokenized_test = testing_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Set format for PyTorch
tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask'])
tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask'])

In [ ]:
# Initialize data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Disable Masked Language Modeling for GPT-2
)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',                # Output directory
    num_train_epochs=3,                    # Number of training epochs
    per_device_train_batch_size=4,         # Batch size for training
    per_device_eval_batch_size=4,          # Batch size for evaluation
    evaluation_strategy='epoch',           # Evaluation strategy
    save_strategy='epoch',                 # Save strategy
    logging_dir='./logs',                  # Directory for logs
    logging_steps=10,                      # Log every 10 steps
    save_total_limit=2,                    # Limit the total amount of checkpoints
    remove_unused_columns=True,            # Remove columns not used by the model
    push_to_hub=False,                     # Disable pushing to hub
    report_to='none',                      # Disable reporting to hub
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,                           # The pre-trained model
    args=training_args,                    # Training arguments
    train_dataset=tokenized_train,         # Training dataset
    eval_dataset=tokenized_test,           # Evaluation dataset
    data_collator=data_collator,           # Data collator
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.495900,3.050524
2,3.061400,2.790959
3,2.911700,2.704385


TrainOutput(global_step=750, training_loss=3.2257609761555988, metrics={'train_runtime': 180.494, 'train_samples_per_second': 16.621, 'train_steps_per_second': 4.155, 'total_flos': 195969024000000.0, 'train_loss': 3.2257609761555988, 'epoch': 3.0})

In [ ]:
testing['text'][422]

'<s>[INST] Which deep learning model performed better? [/INST] No To evaluate the performance of the proposed approach, precision (1), recall (2), f-Measure (3), and prediction accuracy (4) have been used as a performance matrices. The experimental results are shown in Table 1, where it can be seen that autoencoders outperformed MLP and CNN outperformed autoencoders with the highest achieved accuracy of 82.6%. DISPLAYFORM0 DISPLAYFORM1 </s>'

In [ ]:
# Generate sample text
prompt = "which model achive this accuracy 82.6%?"
inputs = tokenizer.encode(prompt, return_tensors='pt').to(model.device)
outputs = model.generate(inputs, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

which model achive this accuracy 82.6%? [/s] No We compare our model with the following two models: the BIBREF0 and BIBREF1. The BIBREF1 is a multi-task model with a
